## Import of the libraries

In [1]:
import numpy as np
import pickle
from keras.models import Sequential
from keras import layers
from keras.optimizers import Adam
from keras import backend as K
import tensorflow as tf
from numpy.random import RandomState

Using TensorFlow backend.


## Function to perform the forecast once the network is trained

In [ ]:
def forecast_cluster(model, data, min_index, max_index, delay, lookback):
    load_forecasted = np.zeros((max_index - (min_index + lookback), 1))
    for idx, i in enumerate(range(min_index+lookback, max_index)):
        block = data[i-lookback:i].reshape((1, lookback, data.shape[1], data.shape[2], 1))
        load_forecasted[idx] = model.predict(block)
    return load_forecasted

## $\alpha$-OMC loss function

In [ ]:
def cost_func_more_args(alpha):
    def cost_func(y_true, y_pred):
        epsilon = 0.1
        diff = y_pred - y_true
        cost = np.zeros(diff.shape)
        y1 = -epsilon * diff + alpha
        y2 = -np.true_divide(1, epsilon) * diff + alpha
        y3 = np.true_divide(alpha, 1-(epsilon*alpha)) * (diff - (epsilon*alpha)) 
        cost = tf.where(diff > (epsilon*alpha), y3, cost)
        cost = tf.where(diff < 0, y1, cost)
        cost = tf.where(tf.logical_and((diff <= (epsilon*alpha)), (diff >= 0)), y2, cost)
        cost = K.mean(cost, axis=-1)
        return cost
    return cost_func

## Build DeepCog model

In [7]:
def make_nn_model(load_input_shape, lookback, num_cluster):
    '''Build DeepCog architecture'''
    model = Sequential()
    model.add(layers.Conv3D(32, kernel_size=(3,3,3), activation='relu', input_shape=(lookback, load_input_shape[1], load_input_shape[2], 1), padding='same'))
    model.add(layers.Conv3D(32, kernel_size=(6,6,6), activation='relu', padding='same'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Conv3D(16, kernel_size=(6,6,6), activation='relu', padding='same'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(num_cluster))
    model.compile(optimizer=Adam(0.0005), loss=cost_func(alpha))
    return model 

## Create, training and forecast with DeepCog 

In [8]:
alpha = 2
lookback = 6
batch_size = 128
num_cluster = 1
load_input # numpy matrix of shape (len(dataset), nº row, nº col, channel). In DeepCog channel = 1
load_output # numpy matrix of shape (len(dataset), num_cluster, channel). In DeepCog channel = 1
nn = make_nn_model(agg_load.shape, lookback, num_cluster)
history = nn.fit_generator(train_gen, steps_per_epoch=150, epochs=100, validation_data=val_gen, validation_steps=40, verbose=0)
load_forecasted_corr = forecast_cluster(nn, load_input, min_index=start_index_test_dataset, max_index=end_index_test_dataset, delay=delay, lookback=lookback)